In [ ]:
import time
import requests
from datetime import datetime

BASE_URL = "https://api.mangadex.org/manga"
RESULTS = []

# Cấu hình
MIN_DELAY = 0.25
delay = 0.0  # ban đầu spam liên tục
MAX_RETRIES = 5
BURST_SUCCESS_THRESHOLD = 8  # số lần thành công liên tiếp để quay lại spam
success_streak = 0

def fetch_all_manga():
    global delay, success_streak
    limit = 100
    created_at_since = "1900-01-01T00:00:00"  # lấy từ đầu

    while True:
        params = {
            "limit": limit,
            "order[createdAt]": "asc",
            "createdAtSince": created_at_since,
            "includes[]": ["author", "artist", "cover_art"]
        }

        retries = 0
        success = False

        while not success and retries < MAX_RETRIES:
            try:
                resp = requests.get(BASE_URL, params=params, timeout=30)

                if resp.status_code == 200:
                    batch = resp.json()
                    data_list = batch.get("data", [])

                    if not data_list:
                        print("🏁 Hoàn tất!")
                        return RESULTS

                    for manga in data_list:
                        RESULTS.append(manga)

                    last_created = data_list[-1]["attributes"]["createdAt"]
                    created_at_since = _add_one_second(last_created)

                    print(f"✅ Lấy {len(data_list)} manga, tổng cộng: {len(RESULTS)} | delay={delay:.2f}s")

                    # Burst mode logic
                    success_streak += 1
                    if delay > 0:  # đang trong chế độ delay
                        delay = max(MIN_DELAY, delay * 0.9)  # giảm delay dần
                        if success_streak >= BURST_SUCCESS_THRESHOLD:
                            delay = 0.0  # quay lại spam mode
                            print("🚀 Quay lại SPAM MODE")
                    success = True

                else:
                    raise Exception(f"HTTP {resp.status_code}")

            except Exception as e:
                retries += 1
                success_streak = 0
                # bật chế độ delay adaptive khi gặp lỗi
                if delay == 0.0:
                    delay = MIN_DELAY
                delay *= 2
                print(f"⚠️ Lỗi: {e}. Retry {retries}/{MAX_RETRIES}, delay={delay:.2f}s")
                time.sleep(delay)

        if not success:
            print(f"❌ Dừng vì lỗi liên tục tại mốc {created_at_since}")
            break

        if delay > 0:
            time.sleep(delay)  # áp dụng delay khi không ở spam mode

def _add_one_second(timestr):
    """Tăng thêm 1 giây cho mốc thời gian ISO8601"""
    dt = datetime.fromisoformat(timestr.replace("Z", "+00:00"))
    dt_plus = dt.timestamp() + 1
    return datetime.utcfromtimestamp(dt_plus).strftime("%Y-%m-%dT%H:%M:%S")

In [ ]:
# Chạy crawler
all_manga = fetch_all_manga()

# Kiểm tra nhanh
print(f"Thu được {len(all_manga)} manga")
all_manga[0]  # xem thử phần tử đầu tiên

In [ ]:
import json

with open("manga_data.json", "w", encoding="utf-8") as f:
    json.dump(all_manga, f, ensure_ascii=False, indent=2)

print("💾 Đã lưu ra manga_data.json")

In [14]:
import time
import requests
from datetime import datetime
from pymongo import MongoClient

In [20]:
# ===== MongoDB setup =====
mongo_client = MongoClient("mongodb://localhost:27017/")
db = mongo_client["manga_raw_data"]
collection = db["mangadex_manga"]

In [21]:
# ===== API config =====
BASE_URL = "https://api.mangadex.org/manga"
MIN_DELAY = 0.25
delay = 0.0
MAX_RETRIES = 5
BURST_SUCCESS_THRESHOLD = 8
success_streak = 0

In [22]:
def fetch_all_manga():
    global delay, success_streak
    limit = 100
    created_at_since = "1900-01-01T00:00:00"

    while True:
        params = {
            "limit": limit,
            "order[createdAt]": "asc",
            "createdAtSince": created_at_since,
            "includes[]": ["author", "artist", "cover_art"]
        }

        retries = 0
        success = False

        while not success and retries < MAX_RETRIES:
            try:
                resp = requests.get(BASE_URL, params=params, timeout=30)

                if resp.status_code == 200:
                    batch = resp.json()
                    data_list = batch.get("data", [])

                    if not data_list:
                        print("🏁 Hoàn tất!")
                        return

                    # lưu từng manga vào MongoDB
                    for manga in data_list:
                        collection.insert_one(manga)

                    last_created = data_list[-1]["attributes"]["createdAt"]
                    created_at_since = _add_one_second(last_created)

                    total_count = collection.count_documents({})
                    print(f"✅ Lấy {len(data_list)} manga, tổng cộng trong DB: {total_count} | delay={delay:.2f}s")

                    # Burst mode logic
                    success_streak += 1
                    if delay > 0:
                        delay = max(MIN_DELAY, delay * 0.9)
                        if success_streak >= BURST_SUCCESS_THRESHOLD:
                            delay = 0.0
                            print("🚀 Quay lại SPAM MODE")
                    success = True

                else:
                    raise Exception(f"HTTP {resp.status_code}")

            except Exception as e:
                retries += 1
                success_streak = 0
                if delay == 0.0:
                    delay = MIN_DELAY
                delay *= 2
                print(f"⚠️ Lỗi: {e}. Retry {retries}/{MAX_RETRIES}, delay={delay:.2f}s")
                time.sleep(delay)

        if not success:
            print(f"❌ Dừng vì lỗi liên tục tại mốc {created_at_since}")
            break

        if delay > 0:
            time.sleep(delay)

In [23]:
def _add_one_second(timestr):
    dt = datetime.fromisoformat(timestr.replace("Z", "+00:00"))
    dt_plus = dt.timestamp() + 1
    return datetime.utcfromtimestamp(dt_plus).strftime("%Y-%m-%dT%H:%M:%S")

In [24]:
# Chạy để fetch và lưu thẳng vào MongoDB
fetch_all_manga()

C:\Users\Huy\AppData\Local\Temp\ipykernel_28164\2656471103.py:4: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  return datetime.utcfromtimestamp(dt_plus).strftime("%Y-%m-%dT%H:%M:%S")


✅ Lấy 100 manga, tổng cộng trong DB: 100 | delay=0.00s
✅ Lấy 100 manga, tổng cộng trong DB: 200 | delay=0.00s
✅ Lấy 100 manga, tổng cộng trong DB: 300 | delay=0.00s
✅ Lấy 100 manga, tổng cộng trong DB: 400 | delay=0.00s
✅ Lấy 100 manga, tổng cộng trong DB: 500 | delay=0.00s
✅ Lấy 100 manga, tổng cộng trong DB: 600 | delay=0.00s
✅ Lấy 100 manga, tổng cộng trong DB: 700 | delay=0.00s
✅ Lấy 100 manga, tổng cộng trong DB: 800 | delay=0.00s
✅ Lấy 100 manga, tổng cộng trong DB: 900 | delay=0.00s
✅ Lấy 100 manga, tổng cộng trong DB: 1000 | delay=0.00s
✅ Lấy 100 manga, tổng cộng trong DB: 1100 | delay=0.00s
✅ Lấy 100 manga, tổng cộng trong DB: 1200 | delay=0.00s
✅ Lấy 100 manga, tổng cộng trong DB: 1300 | delay=0.00s
✅ Lấy 100 manga, tổng cộng trong DB: 1400 | delay=0.00s
✅ Lấy 100 manga, tổng cộng trong DB: 1500 | delay=0.00s
✅ Lấy 100 manga, tổng cộng trong DB: 1600 | delay=0.00s
✅ Lấy 100 manga, tổng cộng trong DB: 1700 | delay=0.00s
✅ Lấy 100 manga, tổng cộng trong DB: 1800 | delay=0.00s
✅

In [25]:
old_db = mongo_client["mangadex_raw_data"]
old_collection = old_db["mangadex_manga"]

new_db = mongo_client["manga_raw_data"]
new_collection = new_db["mangadex_manga"]

In [26]:
# Lấy toàn bộ dữ liệu từ collection cũ
docs = list(old_collection.find({}))

if docs:
    # Xóa _id cũ để MongoDB tự tạo lại (tránh trùng _id)
    for d in docs:
        d.pop("_id", None)

    # Insert sang DB mới
    new_collection.insert_many(docs)
    print(f"✅ Đã copy {len(docs)} document sang 'manga_raw_data.mangadex_manga'")

    # (Tuỳ chọn) Xoá collection cũ
    # old_collection.drop()

✅ Đã copy 86862 document sang 'manga_raw_data.mangadex_manga'


In [ ]:
import json
from pymongo import MongoClient

# Kết nối MongoDB
mongo_client = MongoClient("mongodb://localhost:27017/")
db = mongo_client["mangadex_raw_data"]
collection = db["manga"]

# Đọc dữ liệu từ file
with open("manga_data.json", "r", encoding="utf-8") as f:
    manga_list = json.load(f)

# Lưu vào MongoDB
if isinstance(manga_list, list):
    collection.insert_many(manga_list)
else:
    print("⚠️ File không chứa list!")

print(f"✅ Đã insert {collection.count_documents({})} tài liệu vào MongoDB.")